# ETL Project

## Team: ETheL 

### Neil Patel, Caroline Miller, Ashley Fay

In [6]:
#Required imports

import pandas as pd
import numpy as np
import datetime as datetime
import json
from pandas.io.json import json_normalize 

In [35]:
#Load first data file, Amazon

file_1 = "amazon_com.csv"
amazon = pd.read_csv(file_1)
#amazon.head(2)

In [43]:
#Load second data file, Amazon with extras

file_2 = "amazon_com_extras.csv"
amazon_ex = pd.read_csv(file_2, encoding = "ISO-8859-1")

#Select columns desired for usable df
amazon_ex = amazon_ex[['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']]

amazon_ex.rename(columns={'TITLE':'book_title'}, inplace=True)

#Make column headers lowercase
amazon_ex.columns = amazon_ex.columns.str.lower()

amazon_ex.head(2)

,asin,group,format,book_title,author,publisher
0,1594391297,book,paperback,The Little Black Book of Violence: What Every ...,"Lawrence A. Kane, Kris Wilder, Lt. Col. John R...",Rory Miller
1,761185526,book,paperback,Prison Ramen: Recipes and Stories from Behind ...,"Clifton Collins Jr., Gustavo Goose"" Alvarez",Samuel L. Jackson


In [44]:
#Load third data file, Netflix movies

file_3 = "netflix_movie_titles.csv"

#Apply encoding and code for file errors
netflix = pd.read_csv(file_3, encoding = "ISO-8859-1", header = None, engine = 'python', error_bad_lines=False)

#Rename column names from index based
netflix.rename(columns={0:'Count',1:'year', 2:'netflix_title'}, inplace=True)


#Drop count
netflix.drop(columns=['Count'], inplace = True)

netflix.tail(2)

,year,netflix_title
17768,2003.0,The Company
17769,2003.0,Alien Hunter


In [45]:
#Netflix file - Remove NaN and replace with 0 in the Year column
netflix['year'].fillna(0, inplace = True)

#Change column from float to int
netflix['year'].astype(int).astype(str)

netflix.dtypes

year             float64
netflix_title     object
dtype: object

In [46]:
#Load fourth file, Movies_Metadata

file_4 = "movies_metadata.csv"

#Apply encoding and code for file errors
movies = pd.read_csv(file_4, encoding = "ISO-8859-1", engine = 'python', error_bad_lines=False)

#movies.head()

In [47]:
#Drop columns from the movies file not being used
movies.drop(columns=['adult','budget','homepage','popularity','id','production_countries','imdb_id','original_language','original_title','poster_path','production_companies','belongs_to_collection','runtime','revenue','overview','spoken_languages','video','vote_count'], inplace = True)
#movies.head()

In [48]:
#Merge movies file and amazon books file into 1 df by title
#merged_df = pd.merge(amazon_ex, movies, on='title', how = 'inner')

merged_df = pd.merge(amazon_ex, movies, left_on='book_title',right_on= 'title')

#Make title column the index
#merged_df = merged_df.set_index('title')

merged_df.head(2)

,asin,group,format,book_title,author,publisher,genres,release_date,status,tagline,title,vote_average
0,1405246413,book,hardcover,Go,Diego,Go! Annual 2010,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",1999-04-09,Released,A weekend wasted is never a wasted weekend.,Go,7.0
1,1498756301,book,hardcover,Faster,Better,Cheaper in the History of Manufacturing: From...,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",2010-11-23,Released,Slow Justice is no Justice,Faster,6.1


In [50]:
#Now merge Netflix file into df 
merged_df_2 = pd.merge(merged_df, netflix, left_on='title',right_on= 'netflix_title')
merged_df_2.head(2)

,asin,group,format,book_title,author,publisher,genres,release_date,status,tagline,title,vote_average,year,netflix_title
0,1405246413,book,hardcover,Go,Diego,Go! Annual 2010,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",1999-04-09,Released,A weekend wasted is never a wasted weekend.,Go,7.0,1999.0,Go
1,1405246413,book,hardcover,Go,Diego,Go! Annual 2010,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",1999-04-09,Released,A weekend wasted is never a wasted weekend.,Go,7.0,1969.0,Go


In [52]:
#Drop duplicate titles from merged_df_2, keep first
merged_df_2.drop_duplicates('title', keep = 'first', inplace=True)
merged_df_2.tail(1)

,asin,group,format,book_title,author,publisher,genres,release_date,status,tagline,title,vote_average,year,netflix_title
1488,1935460900,book,paperback,Cabin Fever,James M. Jackson,Barking Rain Press,"[{'id': 27, 'name': 'Horror'}]",2003-08-15,Released,Terrorâ¦ in the flesh.,Cabin Fever,5.5,2003.0,Cabin Fever


In [53]:
#Pull out title and genres column of merged_df_2 to work with dictionaries ect within the column
#Create new smaller df to work with
genre_df = merged_df_2[['title','genres']]

#Reset the index
genre_df = genre_df.reset_index(drop=True)

genre_df.head(1)

,title,genres
0,Go,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name..."


In [54]:
#Create itterator that adjusts for strings by taking length of frame throught the index.
d = {'id': [], 'name': [], 'title': []}
genre_df_out = pd.DataFrame()
for index in range(len(genre_df.genres)):
    item = genre_df.genres[index]
    title = genre_df.title[index]
    #print (title)
    dataItem = json.loads(item.replace("'",'"'))
    
    df_genre = pd.DataFrame(data=dataItem)
    new_bucket = []
    tempName = ''
    genre1 = ''
    genre2 = ''
    genre3 =''
    genre4 = ''
    tempID = ''
    for index2, row in df_genre.iterrows():
        #print(row["id"],row["name"],title)
        #name = name + row["name"]
        tempName = tempName + str(row["name"]) + ','
        tempID = tempID + str(row["id"]) + ','
        if(index2 == 0):genre1 = row["name"]
        if(index2 == 1):genre2 = row["name"]
        if(index2 == 2):genre3 = row["name"]
        if(index2 == 3):genre4 = row["name"]
    
    
    new_bucket.append({'id': tempID, 'genre1': genre1,'genre2': genre2,'genre3': genre3,'genre4': genre4, 'title': title})
    genre_df_out = genre_df_out.append(new_bucket)
    
    #print(genre_df_out)   


In [55]:
#Show final output of original genre column
genre_df_out.head(3)

,genre1,genre2,genre3,genre4,id,title
0,Crime,Comedy,Thriller,,"80,35,53,",Go
0,Crime,Drama,Action,Thriller,"80,18,28,53,",Faster
0,Comedy,Drama,Romance,Music,"35,18,10749,10402,",Girl


In [56]:
#Merge output back with main df (merged_df_2)
final_df = pd.merge(merged_df_2, genre_df_out, on='title', how = 'inner')
#Drop Duplicates
final_df.drop_duplicates('title', keep = 'first', inplace = True)
#Drop initial genres column
final_df.drop(columns=['genres','id'], inplace = True)
#Rename title to book title
final_df = final_df.rename(columns = {'title': 'book title'})
#Set all columns to lowercase
final_df.columns = final_df.columns.str.lower()

#Reset Index
final_df = final_df.reset_index(drop=True)
final_df.tail()

,asin,group,format,book_title,author,publisher,release_date,status,tagline,book title,vote_average,year,netflix_title,genre1,genre2,genre3,genre4
440,1450232876,book,paperback,The Good Thief,Barry Connolly,iUniverse.com,2003-04-03,Released,He doesn't want money. He wants what money can...,The Good Thief,6.0,2003.0,The Good Thief,Crime,Drama,Thriller,
441,198143433X,book,paperback,Along Came a Spider,Michael Joseph Bacotti Sr,CreateSpace Independent Publishing Platform,2001-04-06,Released,The game is far from over.,Along Came a Spider,6.1,2001.0,Along Came a Spider,Crime,Mystery,Thriller,Action
442,679772677,book,paperback,A Civil Action,Jonathan Harr,Vintage,1998-03-05,Released,Justice has its price.,A Civil Action,6.1,1998.0,A Civil Action,Drama,,,
443,B000JGQRPC,book,paperback,Bridget Jones's Diary,Helen Fielding,Picador,2001-04-13,Released,Health Warning: Adopting Bridget's lifestyle c...,Bridget Jones's Diary,6.6,2001.0,Bridget Jones's Diary,Comedy,Romance,Drama,
444,1935460900,book,paperback,Cabin Fever,James M. Jackson,Barking Rain Press,2003-08-15,Released,Terrorâ¦ in the flesh.,Cabin Fever,5.5,2003.0,Cabin Fever,Horror,,,


In [57]:
#Rename columns to remove spaces
final_df.rename(columns={'book title':'book_title','movie title':'movie_title'}, inplace=True)

In [58]:
#Write file to CSV
final_df.to_csv('master_data.csv', index = False)
